# Data Preparation

## Import libraries

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import openai 
from dotenv import load_dotenv
import shutil
import os
import csv

## Load the data

In [10]:
DATA_PATH = "./data/TravelDataset.csv" # Assuming your CSV is directly in this 'data' directory

def read_city_information(headers, row):
    # Combine headers with row values to create meaningful content
    content_parts = []
    row_data = {}
    for header, value in zip(headers, row):
        header_name = header.replace("_", " ")
        content_parts.append(f"{header_name}: {value}")
        row_data[header] = value # Store for easy access to city name
    
    page_content = "\n".join(content_parts)
    return page_content, row_data

def load_and_process_csv_into_documents(file_path: str) -> list[Document]:
    """
    Loads a CSV file, processes each row, and creates LangChain Document objects.
    Each document will represent one city (row), with column names included in the content
    and city name in metadata.
    """
    documents = []
    headers = []

    with open(file_path, 'r', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)

        # Read the header row
        try:
            headers = next(csv_reader)
        except StopIteration:
            print(f"Warning: CSV file '{file_path}' is empty.")
            return []

        # Process each data row
        for i, row in enumerate(csv_reader):
            page_content, row_data = read_city_information(headers, row)

            # Extract city name for metadata (assuming 'city' is one of your columns)
            city_name = row_data.get('city', 'Unknown City')
            document_id = row_data.get('id', None) # Assuming 'id' is a unique identifier

            # Create the Document object
            metadata = {
                "source_file": os.path.basename(file_path), # Name of the CSV file
                "row_number": i + 2, # +2 because of 0-indexed loop and 1-indexed header
                "city_name": city_name,
                "document_type": "city_data"
            }
            if document_id:
                metadata["id"] = document_id # Add the ID if it exists and is useful

            doc = Document(
                page_content=page_content,
                metadata=metadata
            )
            documents.append(doc)

    return documents

In [11]:
processed_documents = load_and_process_csv_into_documents(DATA_PATH)

In [12]:
def display_documents_info(documents: list[Document], num_to_display: int = 3):
    """
    Prints information about the generated documents (chunks).
    """
    print(f"Total documents (chunks) created: {len(documents)}")
    for i, doc in enumerate(documents):
        if i >= num_to_display:
            break
        print(f"\n--- Chunk {i+1} ---")
        print(f"Content:\n{doc.page_content}")
        print(f"Metadata: {doc.metadata}")
    if len(documents) > num_to_display:
        print(f"\n... (displaying first {num_to_display} chunks only)")


In [13]:
display_documents_info(processed_documents, num_to_display=3)

Total documents (chunks) created: 560

--- Chunk 1 ---
Content:
id: c54acf38-3029-496b-8c7a-8343ad82785c
city: Milan
country: Italy
region: europe
short description: Chic streets lined with fashion boutiques, historic architecture, and lively piazzas create a sophisticated yet welcoming atmosphere, perfect for leisurely exploration.
latitude: 45.4641943
longitude: 9.1896346
avg temp monthly: {"1":{"avg":3.7,"max":7.8,"min":0.4},"2":{"avg":7.1,"max":12,"min":2.8},"3":{"avg":10.5,"max":15.5,"min":5.5},"4":{"avg":13.8,"max":18.9,"min":8.7},"5":{"avg":17.9,"max":22.5,"min":13.4},"6":{"avg":23.5,"max":28.5,"min":18.1},"7":{"avg":25.8,"max":30.8,"min":20.5},"8":{"avg":25.2,"max":30.4,"min":20.2},"9":{"avg":20.8,"max":26,"min":16.1},"10":{"avg":15.2,"max":19.6,"min":11.5},"11":{"avg":8.8,"max":12.5,"min":5.6},"12":{"avg":4.7,"max":8.2,"min":1.9}}
ideal durations: ["Short trip","One week"]
budget level: Luxury
culture: 5
adventure: 2
nature: 2
beaches: 1
nightlife: 4
cuisine: 5
wellness: 3
urb

## Create vector database

In [14]:
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [15]:
embedding_model = "text-embedding-3-small"
embedding_function = OpenAIEmbeddings(model=embedding_model)

In [16]:
CHROMA_PATH = "chroma"

In [17]:
# Clear the previous version of the database
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

if not os.path.exists(CHROMA_PATH):
    os.makedirs(CHROMA_PATH)

In [18]:
db = Chroma.from_documents(
    processed_documents, embedding_function, persist_directory=CHROMA_PATH
)
print(f"Saved {len(processed_documents)} documents to the database {CHROMA_PATH}")

## Query relevant data

In [21]:
# load the database
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

In [27]:
query_text = "What are the most touristic destinations?"

In [28]:
results = db.similarity_search_with_relevance_scores(query_text, k=3)


In [33]:
if len(results) == 0:
    print("Unable to find any matching results")
else:
    print("Results:\n")
    for result in results:
        print(f"score: {result[1]}\n, the city: {result[0]}\n--------\n")

Results:

score: 0.14288719016657125
, the city: page_content='id: 577b9315-b7a1-489f-9f37-c705858b1162
city: London
country: United Kingdom
region: europe
short description: Iconic landmarks, bustling markets, and a rich tapestry of culture create a vibrant atmosphere that feels both historic and ever-evolving.
latitude: 51.4893335
longitude: -0.14405508452768728
avg temp monthly: {"1":{"avg":4.6,"max":7.3,"min":1.9},"2":{"avg":6.1,"max":8.8,"min":3.4},"3":{"avg":8.1,"max":12,"min":4.4},"4":{"avg":9.6,"max":13.8,"min":5.6},"5":{"avg":13,"max":17.4,"min":8.6},"6":{"avg":16.5,"max":21,"min":11.9},"7":{"avg":18.2,"max":22.7,"min":13.6},"8":{"avg":17.7,"max":22.1,"min":13.5},"9":{"avg":15.2,"max":19.3,"min":11.3},"10":{"avg":12,"max":15.1,"min":9},"11":{"avg":7.9,"max":10.4,"min":5.3},"12":{"avg":6.9,"max":8.9,"min":4.8}}
ideal durations: ["Long trip","One week","Short trip"]
budget level: Luxury
culture: 5
adventure: 2
nature: 3
beaches: 1
nightlife: 5
cuisine: 5
wellness: 3
urban: 5
sec